<a href="https://colab.research.google.com/github/oorja1/CTTC_AI_ML/blob/main/dog_vs_cat_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 98% 1.04G/1.06G [00:05<00:00, 155MB/s]
100% 1.06G/1.06G [00:05<00:00, 206MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [7]:
#generator
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256, 256),
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256, 256),
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [8]:
#Normalize - to map the pixel values in (0-1)
def process(image,label):
  image = tf.cast(image/255. , tf.float32)
  return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [9]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding = 'valid',activation= 'relu', input_shape = (256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides= 2,padding = 'valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding = 'valid',activation= 'relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides= 2,padding = 'valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding = 'valid',activation= 'relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides= 2,padding = 'valid'))

model.add(Flatten())

model.add(Dense(128,activation= 'relu'))
model.add(Dense(64,activation= 'relu'))
model.add(Dense(1,activation= 'sigmoid'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])


In [12]:
history = model.fit(train_ds,epochs=10,validation_data= validation_ds)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 53s 73ms/step - accuracy: 0.6365 - loss: 0.6519 - val_accuracy: 0.7256 - val_loss: 0.5430
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 76s 74ms/step - accuracy: 0.7731 - loss: 0.4709 - val_accuracy: 0.7940 - val_loss: 0.4530
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.8360 - loss: 0.3649 - val_accuracy: 0.8074 - val_loss: 0.4488
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 47s 75ms/step - accuracy: 0.8974 - loss: 0.2444 - val_accuracy: 0.7894 - val_loss: 0.7540
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 75ms/step - accuracy: 0.9432 - loss: 0.1421 - val_accuracy: 0.8096 - val_loss: 0.7726
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 77s 68ms/step - accuracy: 0.9661 - loss: 0.0919 - val_accuracy: 0.8030 - val_loss: 1.0518
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 86s 75ms/step - accuracy: 0.9780 - loss: 0.0628 - val_accuracy: 0.7884 - val_loss: 1.2167
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 75ms/step - accuracy: 0.9820 - loss: 0.0521 - 